In [1]:
import anndata
from scdesigner.minimal import NegBinCopula

example_sce = anndata.read_h5ad("data/example_sce.h5ad")
sim = NegBinCopula(mean_formula="~ bs(pseudotime, df=5)", dispersion_formula="~ 1", copula_formula="~ -1 + cell_type")
sim.fit(example_sce, max_epochs=1, lr=0.01)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/miniconda3/envs/scdesign/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default

  | Name  | Type          | Params | Mode 
------------------------------------------------
0 | coefs | ParameterDict | 700    | train
-

Epoch 0: 100%|██████████| 3/3 [00:00<00:00,  3.05it/s, v_num=61, train_loss_step=1.77e+4, train_loss_epoch=4.57e+5]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 3/3 [00:00<00:00,  3.03it/s, v_num=61, train_loss_step=1.77e+4, train_loss_epoch=4.57e+5]


Estimating copula covariance: 100%|██████████| 3/3 [00:01<00:00,  2.70it/s]


In [2]:
print(sim.parameters)
print(sim.sample())
print(sim.predict())

{'marginal': {'mean':                               Pyy      Iapp      Chgb      Rbp4      Spp1  \
Intercept               -0.009565  0.009141 -0.004560  0.012290  0.008068   
bs(pseudotime, df=5)[1] -0.013217 -0.013545  0.007277 -0.005120  0.005247   
bs(pseudotime, df=5)[2]  0.004210  0.015174 -0.000185  0.000317  0.012451   
bs(pseudotime, df=5)[3]  0.013855  0.000304 -0.001720 -0.003906 -0.001333   
bs(pseudotime, df=5)[4] -0.016039  0.004896  0.001560  0.005355 -0.016459   
bs(pseudotime, df=5)[5] -0.003506  0.000485  0.015410 -0.013294  0.001127   

                             Chga       Cck      Ins1      Nnat      Ins2  \
Intercept                0.018339  0.003379  0.012522  0.004024 -0.004228   
bs(pseudotime, df=5)[1] -0.009119 -0.006921  0.013733  0.015792  0.003029   
bs(pseudotime, df=5)[2] -0.012471 -0.015242 -0.013854  0.003075  0.005745   
bs(pseudotime, df=5)[3]  0.008857  0.004221  0.000744  0.005812  0.002622   
bs(pseudotime, df=5)[4] -0.012062  0.002990 -0.003979

In [3]:
from scdesigner.minimal import ZeroInflatedNegBinCopula

sim = ZeroInflatedNegBinCopula("~ bs(pseudotime, df=5)", "~ 1", "~ pseudotime", "~ -1 + cell_type")
sim.fit(example_sce, max_epochs=1, lr=0.01)

/Users/malangtian/Documents/Research_Projects/scDesigner_project/scDesigner/scdesigner/src/scdesigner/minimal/formula.py:17: UserWarning: Invalid formulas in dictionary will not be used: {'zero_inflation_formula'}
  warnings.warn(
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type          | Params | Mode 
------------------------------------------------
0 | coefs | ParameterDict | 1.0 K  | train
------------------------------------------------
1.0 K     Trainable params
0         Non-trainable params
1.0 K     Total params
0.004     Total estimated model params size (MB)
1         Modules in train mode
0         Modules in eval mode
/opt/miniconda3/envs/scdesign/lib/python3.12/site-packages/pytorch_ligh

Epoch 0: 100%|██████████| 3/3 [00:01<00:00,  2.91it/s, v_num=62, train_loss_step=1.52e+4, train_loss_epoch=3.8e+5]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 3/3 [00:01<00:00,  2.89it/s, v_num=62, train_loss_step=1.52e+4, train_loss_epoch=3.8e+5]


Estimating copula covariance: 100%|██████████| 3/3 [00:01<00:00,  2.77it/s]


In [4]:
print(sim.parameters)
print(sim.sample())
print(sim.predict())

{'marginal': {'mean':                               Pyy      Iapp      Chgb      Rbp4      Spp1  \
Intercept                0.001981  0.011890  0.003649  0.006014  0.000549   
bs(pseudotime, df=5)[1] -0.009322 -0.008060 -0.013590 -0.011831 -0.000037   
bs(pseudotime, df=5)[2]  0.007936  0.012786  0.012447  0.006394  0.005443   
bs(pseudotime, df=5)[3] -0.007681  0.009492  0.003246 -0.017035  0.006656   
bs(pseudotime, df=5)[4] -0.018146  0.016273 -0.003790 -0.008745 -0.002739   
bs(pseudotime, df=5)[5] -0.013848 -0.004812 -0.003192  0.009810 -0.010249   

                             Chga       Cck      Ins1      Nnat      Ins2  \
Intercept               -0.005817 -0.001509  0.006570 -0.007702  0.001189   
bs(pseudotime, df=5)[1]  0.015024  0.000054 -0.005989  0.013666  0.020528   
bs(pseudotime, df=5)[2] -0.009067  0.000783  0.000857  0.020242 -0.006627   
bs(pseudotime, df=5)[3] -0.007412  0.017478  0.004618  0.005304  0.001404   
bs(pseudotime, df=5)[4] -0.016638 -0.011586 -0.033360

In [5]:
from scdesigner.minimal import BernoulliCopula

sim = BernoulliCopula("~ bs(pseudotime, df=5)", "~ -1 + cell_type")
sim.fit(example_sce, max_epochs=1, lr=0.01)

print(sim.parameters)
print(sim.sample())
print(sim.predict())

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type          | Params | Mode 
------------------------------------------------
0 | coefs | ParameterDict | 700    | train
------------------------------------------------
700       Trainable params
0         Non-trainable params
700       Total params
0.003     Total estimated model params size (MB)
1         Modules in train mode
0         Modules in eval mode
/opt/miniconda3/envs/scdesign/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to imp

Epoch 0: 100%|██████████| 3/3 [00:00<00:00,  3.40it/s, v_num=63, train_loss_step=1.78e+4, train_loss_epoch=4.58e+5]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 3/3 [00:00<00:00,  3.38it/s, v_num=63, train_loss_step=1.78e+4, train_loss_epoch=4.58e+5]


Estimating copula covariance: 100%|██████████| 3/3 [00:01<00:00,  2.79it/s]


{'marginal': {'mean':                               Pyy      Iapp      Chgb      Rbp4      Spp1  \
Intercept               -0.007948  0.004975 -0.003037  0.017223  0.008699   
bs(pseudotime, df=5)[1]  0.009649  0.000913  0.007723 -0.020900 -0.018264   
bs(pseudotime, df=5)[2]  0.005990  0.007039  0.004159  0.015162  0.005133   
bs(pseudotime, df=5)[3] -0.014402  0.017215  0.013362  0.006423  0.004637   
bs(pseudotime, df=5)[4] -0.006864 -0.002093  0.006625  0.003498  0.001759   
bs(pseudotime, df=5)[5] -0.006514  0.031763  0.005759  0.001397 -0.017313   

                             Chga       Cck      Ins1      Nnat      Ins2  \
Intercept                0.000561 -0.001001 -0.001679 -0.007178 -0.003486   
bs(pseudotime, df=5)[1] -0.004087 -0.017794  0.002458 -0.020716 -0.008183   
bs(pseudotime, df=5)[2]  0.004024  0.013371 -0.010805 -0.008856 -0.011805   
bs(pseudotime, df=5)[3]  0.008893  0.004513  0.018472 -0.012213  0.001090   
bs(pseudotime, df=5)[4]  0.002669 -0.002902  0.000239

In [6]:
from scdesigner.minimal import GaussianCopula

sim = GaussianCopula("~ bs(pseudotime, df=5)", "~ pseudotime", "~ -1 + cell_type")
sim.fit(example_sce, max_epochs=1, lr=0.01)

print(sim.parameters)
print(sim.sample())
print(sim.predict())

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type          | Params | Mode 
------------------------------------------------
0 | coefs | ParameterDict | 800    | train
------------------------------------------------
800       Trainable params
0         Non-trainable params
800       Total params
0.003     Total estimated model params size (MB)
1         Modules in train mode
0         Modules in eval mode
/opt/miniconda3/envs/scdesign/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to imp

Epoch 0: 100%|██████████| 3/3 [00:01<00:00,  2.97it/s, v_num=64, train_loss_step=9.02e+5, train_loss_epoch=2.77e+7]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 3/3 [00:01<00:00,  2.96it/s, v_num=64, train_loss_step=9.02e+5, train_loss_epoch=2.77e+7]


Estimating copula covariance: 100%|██████████| 3/3 [00:01<00:00,  2.73it/s]


{'marginal': {'mean':                               Pyy      Iapp      Chgb      Rbp4      Spp1  \
Intercept                0.000088  0.003892 -0.005490 -0.006584 -0.003903   
bs(pseudotime, df=5)[1] -0.000017  0.018998  0.010003  0.012393  0.001135   
bs(pseudotime, df=5)[2] -0.007388 -0.003943 -0.000759  0.003871 -0.007044   
bs(pseudotime, df=5)[3] -0.006034 -0.007559  0.000915 -0.000422 -0.008936   
bs(pseudotime, df=5)[4]  0.005468  0.004962 -0.000174  0.026694  0.004320   
bs(pseudotime, df=5)[5]  0.011270 -0.013021  0.018895 -0.011468 -0.004915   

                             Chga       Cck      Ins1      Nnat      Ins2  \
Intercept                0.002370 -0.010500  0.010706 -0.003109  0.000979   
bs(pseudotime, df=5)[1] -0.010569 -0.000349 -0.002240 -0.009387 -0.024967   
bs(pseudotime, df=5)[2] -0.003755  0.019682 -0.000924  0.000296 -0.011249   
bs(pseudotime, df=5)[3] -0.002246  0.004328  0.010276  0.011641 -0.002233   
bs(pseudotime, df=5)[4]  0.016371  0.013004 -0.022356

/Users/malangtian/Documents/Research_Projects/scDesigner_project/scDesigner/scdesigner/src/scdesigner/minimal/standard_copula.py:404: RuntimeWarning: invalid value encountered in sqrt
  normal_distn = norm(0, np.diag(cov_struct.cov.values) ** 0.5)


AnnData object with n_obs × n_vars = 2087 × 100
    obs: 'clusters_coarse', 'clusters', 'S_score', 'G2M_score', 'cell_type', 'sizeFactor', 'pseudotime'
{'mean': array([[-0.00315765,  0.00016832, -0.00512258, ...,  0.00532763,
        -0.00662933,  0.00888386],
       [-0.00608419,  0.00097008, -0.00466414, ...,  0.00472245,
        -0.00589866,  0.01081529],
       [-0.00071993,  0.00210332, -0.00496609, ...,  0.00405119,
        -0.00941826,  0.00852877],
       ...,
       [-0.00249436,  0.00072895, -0.00512368, ...,  0.00501525,
        -0.00737168,  0.00881966],
       [-0.00511716, -0.00111579, -0.00517188, ...,  0.00563107,
        -0.00516185,  0.00921361],
       [-0.00503447,  0.00582061, -0.00298815, ...,  0.00552584,
        -0.00601082,  0.0131305 ]], dtype=float32), 'sdev': array([[1.0206052 , 1.0009112 , 0.99275804, ..., 1.0057284 , 1.000848  ,
        1.0120817 ],
       [1.0162438 , 0.99788266, 0.9937817 , ..., 1.0029353 , 1.0003083 ,
        1.0154446 ],
       [1.0218

In [7]:
from scdesigner.minimal.negbin import NegBin
from scdesigner.minimal import CompositeCopula

genes = example_sce.var_names
specification = [(genes[:10], NegBin("~ bs(pseudotime, df=5)")), (genes[10:20], NegBin("~ 1"))]
# sim = CompositeCopula(specification, "~ 1")
# sim.fit(example_sce, max_epochs=1)
# sim.predict(example_sce.obs[:10])
# sim.sample(example_sce.obs[:10])
# sim.complexity()

In [8]:
preds = sim.predict(example_sce.obs[:10])

{k: v[:4, :4] for k, v in preds.items()}

{'mean': array([[-3.4208826e-03, -1.7689435e-04, -5.0766910e-03,  3.6786550e-05],
        [-5.8092359e-03,  2.4430293e-03, -4.1557760e-03, -2.5939681e-03],
        [-2.0504289e-04,  2.4337266e-03, -4.8789242e-03,  6.0355566e-03],
        [ 1.1358007e-02, -9.1294087e-03,  1.3404550e-02, -1.8051397e-02]],
       dtype=float32),
 'sdev': array([[1.0206052 , 1.0009112 , 0.99275804, 0.99701333],
        [1.0162438 , 0.99788266, 0.9937817 , 0.99705756],
        [1.0218894 , 1.0018022 , 0.9924576 , 0.99700034],
        [1.0250306 , 1.0039803 , 0.9917249 , 0.9969687 ]], dtype=float32)}

In [9]:
sim.sample(example_sce.obs[:10])

/Users/malangtian/Documents/Research_Projects/scDesigner_project/scDesigner/scdesigner/src/scdesigner/minimal/standard_copula.py:404: RuntimeWarning: invalid value encountered in sqrt
  normal_distn = norm(0, np.diag(cov_struct.cov.values) ** 0.5)


AnnData object with n_obs × n_vars = 10 × 100
    obs: 'clusters_coarse', 'clusters', 'S_score', 'G2M_score', 'cell_type', 'sizeFactor', 'pseudotime'

In [10]:
sim.complexity()

Computing log-likelihood...: 100%|██████████| 3/3 [00:01<00:00,  2.47it/s]


{'aic': 3616501.4976287372, 'bic': 3728242.459188866}

In [11]:
sim.parameters

{'marginal': {'mean':                               Pyy      Iapp      Chgb      Rbp4      Spp1  \
  Intercept                0.000088  0.003892 -0.005490 -0.006584 -0.003903   
  bs(pseudotime, df=5)[1] -0.000017  0.018998  0.010003  0.012393  0.001135   
  bs(pseudotime, df=5)[2] -0.007388 -0.003943 -0.000759  0.003871 -0.007044   
  bs(pseudotime, df=5)[3] -0.006034 -0.007559  0.000915 -0.000422 -0.008936   
  bs(pseudotime, df=5)[4]  0.005468  0.004962 -0.000174  0.026694  0.004320   
  bs(pseudotime, df=5)[5]  0.011270 -0.013021  0.018895 -0.011468 -0.004915   
  
                               Chga       Cck      Ins1      Nnat      Ins2  \
  Intercept                0.002370 -0.010500  0.010706 -0.003109  0.000979   
  bs(pseudotime, df=5)[1] -0.010569 -0.000349 -0.002240 -0.009387 -0.024967   
  bs(pseudotime, df=5)[2] -0.003755  0.019682 -0.000924  0.000296 -0.011249   
  bs(pseudotime, df=5)[3] -0.002246  0.004328  0.010276  0.011641 -0.002233   
  bs(pseudotime, df=5)[4]  0.

In [12]:
import scdesigner.minimal.transform as tr

null_sim = tr.nullify(sim, "pseudotime", "Pyy|Iapp", "mean")
null_sim.marginal.predict.coefs["mean"][:3, :4]

tensor([[ 8.8004e-05,  3.8918e-03, -5.4903e-03, -6.5839e-03],
        [-0.0000e+00,  0.0000e+00,  1.0003e-02,  1.2393e-02],
        [-0.0000e+00, -0.0000e+00, -7.5882e-04,  3.8714e-03]],
       grad_fn=<SliceBackward0>)

In [13]:
sim.parameters["copula"]['cell_type[Ngn3 low EP]'].cov

,Pyy,Iapp,Chgb,Rbp4,Spp1,Chga,Cck,Ins1,Nnat,Ins2,...,Nkx6-1,Fxyd3,Hn1,Smarcd2,Pdia6,Ffar2,Hes6,Serpinh1,Npy,1110012L19Rik
Pyy,0.745184,0.034686,0.026379,0.097484,0.006078,-0.001175,0.029002,0.020458,0.004458,0.000138,...,0.025527,0.013900,0.100598,0.032168,-0.005948,-0.002394,0.141178,0.052187,0.006355,0.039253
Iapp,0.034686,0.304705,-0.018774,0.016291,-0.000217,-0.003401,0.059968,-0.001088,0.002928,0.002633,...,0.069127,0.025622,-0.018492,0.089095,0.038171,0.023508,0.095646,-0.053510,0.001952,-0.010338
Chgb,0.026379,-0.018774,0.085394,0.031161,0.000783,-0.003545,-0.004846,0.003034,0.004286,0.000829,...,-0.004217,0.035424,0.027925,-0.043860,0.025974,-0.025207,-0.016341,-0.000598,-0.000683,0.006573
Rbp4,0.097484,0.016291,0.031161,0.976054,0.005450,-0.005627,0.000390,0.004110,-0.026871,-0.009026,...,0.252270,-0.065093,0.077739,0.056520,0.177359,0.072219,-0.031214,0.040365,-0.004583,-0.028834
Spp1,0.006078,-0.000217,0.000783,0.005450,0.013032,0.000373,-0.006941,0.001727,-0.003734,-0.005090,...,0.007232,-0.003560,-0.000479,-0.005755,0.013398,-0.005888,0.001379,0.015545,0.000052,0.004209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ffar2,-0.002394,0.023508,-0.025207,0.072219,-0.005888,0.005124,0.044060,0.014812,-0.003773,0.032618,...,0.013285,0.041611,0.186032,0.139271,0.014952,0.820620,0.281169,-0.000387,-0.003755,-0.055350
Hes6,0.141178,0.095646,-0.016341,-0.031214,0.001379,0.014312,0.123194,-0.011324,0.031835,0.033423,...,0.067345,0.109144,0.410124,0.395955,0.099048,0.281169,1.730129,0.062466,-0.001717,0.003068
Serpinh1,0.052187,-0.053510,-0.000598,0.040365,0.015545,-0.003924,-0.011388,0.014781,0.038051,0.026584,...,-0.095763,0.078396,0.071107,0.155384,0.493053,-0.000387,0.062466,1.710495,-0.021367,0.120774
Npy,0.006355,0.001952,-0.000683,-0.004583,0.000052,-0.000305,0.003202,-0.001445,-0.001258,-0.000184,...,0.014541,-0.005868,0.005139,-0.005344,-0.003497,-0.003755,-0.001717,-0.021367,0.007599,-0.003444


In [ ]:
import copy

amplified_sim = tr.amplify(sim, 2, "pseudotime", "Pyy|Iapp", "mean")
# decorr_sim = tr.decorrelate(sim, "Pyy|Iapp", "Pyy|Iapp", "cell_type[Ngn3 low EP]")
# corr_sim = tr.correlate(sim, 2, "Pyy|Iapp", "Pyy|Iapp", "cell_type[Ngn3 low EP]")
decorr_sim = copy.deepcopy(sim)
decorr_sim.copula.decorrelate("Pyy|Iapp", "Pyy|Iapp", "cell_type[Ngn3 low EP]")
corr_sim = copy.deepcopy(sim)
corr_sim.copula.correlate(2, "Pyy|Iapp", "Pyy|Iapp", "cell_type[Ngn3 low EP]")

decorr_sim.parameters["copula"]["cell_type[Ngn3 low EP]"].iloc[:4, :4]

AttributeError: 'CovarianceStructure' object has no attribute 'iloc'

Here is a more manual but very general way to substitute parameters.

In [ ]:
import pandas as pd
import numpy as np

new_param = np.random.normal(size=(6, 100))
sub_sim_mean = tr.replace_param(sim, ["marginal", "mean"], new_param)

new_covariance = 0.9 * np.eye(100) + 0.1 * np.ones((100, 100))
new_covariance = pd.DataFrame(new_covariance, columns=example_sce.var_names, index=example_sce.var_names)
sub_sim_cov = tr.replace_param(sim, ["copula", "cell_type[Ngn3 low EP]"], new_covariance)

Here's an example showing how dispersion can be modeled.

In [ ]:
sim = NegBinCopula("~ bs(pseudotime,df=5)", "~ pseudotime", "~ -1 + cell_type")
sim.fit(example_sce, batch_size=int(256), max_epochs=1)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type          | Params | Mode 
------------------------------------------------
0 | coefs | ParameterDict | 800    | train
------------------------------------------------
800       Trainable params
0         Non-trainable params
800       Total params
0.003     Total estimated model params size (MB)
1         Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.
Estimating copula covariance: 100%|██████████| 9/9 [00:01<00:00,  5.21it/s]


In [ ]:
print(sim.parameters)
print(sim.sample())
print(sim.predict())

{'marginal': {'mean':                               Pyy      Iapp      Chgb      Rbp4      Spp1  \
Intercept               -0.004036 -0.005068  0.023052  0.007023 -0.001932   
bs(pseudotime, df=5)[1] -0.002184 -0.014863 -0.017943 -0.010535  0.005288   
bs(pseudotime, df=5)[2]  0.008497  0.005740  0.016936 -0.007517 -0.012623   
bs(pseudotime, df=5)[3]  0.007265  0.003080  0.006423  0.014232 -0.011942   
bs(pseudotime, df=5)[4]  0.010770  0.013250  0.010367  0.000115 -0.015543   
bs(pseudotime, df=5)[5]  0.013108  0.026219  0.002672  0.010289  0.003766   

                             Chga       Cck      Ins1      Nnat      Ins2  \
Intercept                0.014313 -0.004389  0.004286  0.011508  0.020832   
bs(pseudotime, df=5)[1] -0.011925  0.031351  0.008636  0.006924 -0.026107   
bs(pseudotime, df=5)[2]  0.002636  0.017008 -0.014670  0.003025 -0.008852   
bs(pseudotime, df=5)[3] -0.008875  0.009577  0.014945  0.016778  0.015402   
bs(pseudotime, df=5)[4]  0.017718 -0.002636  0.031011

In [ ]:
example_sce = anndata.read_h5ad("data/million_cells.h5ad", backed=True, chunk_size=int(2e4))

sim = NegBinCopula("~ celltype", "~ 1", "~ -1 + celltype")
#sim.fit(example_sce, batch_size=int(1000), lr=0.01)